## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool, set_default_openai_client, set_default_openai_api
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from openai import AsyncAzureOpenAI
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [7]:
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent systems.

**OpenAI Frontier**: Introduced by OpenAI, Frontier is an AI agent management platform designed for enterprise customers. It enables users to build, deploy, and manage AI agents from both OpenAI and third-party developers in a unified environment. Frontier addresses "agent sprawl" by providing a centralized system that supports various operating environments—local, cloud, and OpenAI-hosted—and utilizes open standards for integration. Each AI agent is assigned a unique identity with specific permissions and guardrails, ensuring compliance in regulated industries. ([techradar.com](https://www.techradar.com/pro/openai-introduces-frontier-an-easier-way-to-manage-all-your-ai-agents-in-one-place?utm_source=openai))

**Agent Lightning**: Developed by a team of researchers, Agent Lightning is a flexible and extensible framework that enables Reinforcement Learning (RL)-based training of Large Language Models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents developed through various methods. By formulating agent execution as a Markov decision process, Agent Lightning introduces a hierarchical RL algorithm, LightningRL, which includes a credit assignment module for decomposing trajectories into training transitions. Experiments across tasks like text-to-SQL, retrieval-augmented generation, and math tool-use demonstrate stable, continuous improvements, showcasing the framework's potential for real-world agent training and deployment. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

**Project Mariner**: Developed by Google DeepMind, Project Mariner is a research prototype that explores human-agent interactions within web browsers. It automates tasks such as online shopping, information retrieval, and form-filling, aiming to enhance user productivity by delegating routine web-based tasks to an AI agent. Operating as an experimental Chrome extension, Project Mariner understands the contents of the user's screen, interprets complex goals, plans actionable steps, and navigates websites to carry out tasks, while keeping the user informed and allowing them to intervene at any time. As of May 2025, Project Mariner is available to Google AI Ultra subscribers in the US and is being integrated into the Gemini API and Vertex AI, allowing developers to build applications powered by the agent. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Project_Mariner?utm_source=openai))

**OpenAI Agents SDK**: Released in March 2025, the OpenAI Agents SDK is a lightweight Python framework focusing on creating multi-agent workflows with comprehensive tracing and guardrails. It offers provider-agnostic compatibility with over 100 different LLMs, enabling developers to build and manage complex agent systems efficiently. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

**Google Agent Development Kit (ADK)**: Announced in April 2025, Google's ADK is a modular framework that integrates with the Google ecosystem, including Gemini and Vertex AI. It supports hierarchical agent compositions and requires minimal code for efficient development, facilitating the creation of complex agent systems within Google's infrastructure. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

These frameworks represent significant advancements in AI agent development, each contributing to the evolution of intelligent systems capable of performing complex tasks autonomously. 

### As always, take a look at the trace

https://platform.openai.com/traces

### We will now use Structured Outputs, and include a description of the fields

In [5]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    output_type=WebSearchPlan,
)

In [6]:

message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To identify the most current and popular AI agent frameworks available in 2025.', query='top AI agent frameworks 2025'), WebSearchItem(reason='To find expert analyses, reviews, or tech articles discussing advancements in AI agent frameworks up to 2025.', query='latest advancements in AI agent frameworks 2025'), WebSearchItem(reason='To learn about emerging open-source projects and updates for AI agent frameworks announced or released in 2025.', query='open source AI agent frameworks released 2025')]


In [8]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("daniel1957000@gmail.com") # Change this to your verified email
    to_email = To("dd3269@columbia.edu") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "success"

In [9]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026F35F8C680>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [10]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [11]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    output_type=ReportData,
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [12]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and email it

In [13]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime!

In [14]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")




Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!


In [15]:
print(report)


short_summary='The landscape of AI agent frameworks in 2025 is rapidly evolving, with major players like OpenAI, Google, Microsoft, and Salesforce launching advanced platforms for developing, managing, and deploying autonomous agents. Alongside corporate solutions, open-source frameworks such as Cognitive Kernel-Pro, EnvX, and LightAgent are offering robust, accessible tools for diverse agentic applications. These innovations reflect a shift towards more modular, collaborative, and enterprise-ready AI systems.' markdown_report='# Latest AI Agent Frameworks in 2025\n\n## Table of Contents\n1. Introduction\n2. The Evolution of AI Agent Frameworks\n3. Major Proprietary AI Agent Frameworks (2024–2025)\n    - 3.1 OpenAI’s Frontier\n    - 3.2 Google’s Project Mariner and Antigravity\n    - 3.3 Salesforce’s Agentforce 360\n    - 3.4 Microsoft’s Security Copilot\n4. Leading Open-Source and Community Frameworks\n    - 4.1 Cognitive Kernel-Pro\n    - 4.2 EnvX\n    - 4.3 LightAgent\n5. Popular De

In [16]:
import IPython
from IPython.display import Markdown, display

display(Markdown(report.markdown_report))

# Latest AI Agent Frameworks in 2025

## Table of Contents
1. Introduction
2. The Evolution of AI Agent Frameworks
3. Major Proprietary AI Agent Frameworks (2024–2025)
    - 3.1 OpenAI’s Frontier
    - 3.2 Google’s Project Mariner and Antigravity
    - 3.3 Salesforce’s Agentforce 360
    - 3.4 Microsoft’s Security Copilot
4. Leading Open-Source and Community Frameworks
    - 4.1 Cognitive Kernel-Pro
    - 4.2 EnvX
    - 4.3 LightAgent
5. Popular Developer-Centric Frameworks
    - 5.1 LangChain
    - 5.2 CrewAI
    - 5.3 OpenAI Agents SDK
    - 5.4 Google Agent Development Kit (ADK)
    - 5.5 Microsoft Semantic Kernel
6. Comparative Analysis
7. Current Challenges and Future Trends
8. Conclusion
9. References

---

## 1. Introduction

AI agents—autonomous systems able to perceive, reason, and act on behalf of users—have seen a surge in research and commercial interest. By 2025, a host of proprietary and open-source frameworks have emerged, each designed to enable more powerful, collaborative, and useful intelligent agents. This report presents a comprehensive overview of the latest AI agent frameworks as of 2025, highlighting their features, strengths, and impact on the evolving field of artificial intelligence.

## 2. The Evolution of AI Agent Frameworks

The concept of AI agents has evolved from simple bots to complex, multi-modal systems capable of handling natural language, perceiving environments, and executing goal-driven actions. The proliferation of advanced foundation models, improvements in tool integration, and increased enterprise demand have all accelerated the development and adoption of robust agentic frameworks. In 2024 and 2025, the integration of workflow management, security, traceability, and cross-platform interoperability has become central to agentic design.

## 3. Major Proprietary AI Agent Frameworks (2024–2025)

### 3.1 OpenAI’s Frontier

* **Overview**: Launched in early 2026, Frontier is OpenAI’s enterprise-grade agent management platform. It simplifies building, deploying, and managing AI agents from OpenAI and third parties, offering a unified environment for agent operations.
* **Key Features**:
    - Unifies fragmented tools and data, reducing "agent sprawl"
    - Supports various operating environments (local, cloud, OpenAI-hosted)
    - Open standards and extensive API support
    - Integration with both OpenAI and external agents for flexible development

Frontier addresses significant scaling and governance challenges in AI agent operations, providing a foundation for complex, multi-agent enterprise systems.

### 3.2 Google’s Project Mariner and Antigravity

**Project Mariner**:
* Launched as a Chrome extension in December 2024, now available to US users via Google AI Ultra.
* Automates browser-based tasks—shopping, data entry, and information retrieval—by analyzing web page content and executing actions autonomously.
* Integrated into Gemini API and Vertex AI, allowing developers to build custom applications on top of Mariner.

**Antigravity**:
* A next-generation AI-powered IDE, forked from Visual Studio Code and launched in late 2025.
* Allows developers to delegate aspects of software development to Gemini-powered agents, automating coding, debugging, and even test generation within an intelligent IDE.
* Prioritizes developer productivity and agent collaboration for software engineering.

### 3.3 Salesforce’s Agentforce 360

* Introduced in mid-2025, Agentforce 360 targets enterprise customers with a comprehensive agent management suite.
* Provides:
    - Agentforce Builder: Create, preview, and integrate AI agents using natural language commands
    - API/JSON workflows for flexible integration
    - Agentforce Voice: Real-time speech-driven agent interactions
    - Tight integration with Slack for workflow automation and team collaboration

Salesforce’s focus is on business automation, workflow orchestration, and seamless deployment within enterprise ecosystems.

### 3.4 Microsoft’s Security Copilot

* In March 2025, Microsoft enhanced Security Copilot with 11 new specialized agents to automate routine cybersecurity tasks across their security suite.
* Features include:
    - Automated detection, triage, and alert management
    - Continuous learning from user feedback to correct false positives
    - Broad integration with Microsoft’s security tools

This initiative is part of a broader move towards domain-specific "copilots" for reducing burnout and improving resilience in critical security infrastructure.

## 4. Leading Open-Source and Community Frameworks

### 4.1 Cognitive Kernel-Pro

* **Description**: Cognitive Kernel-Pro is an open-source, multi-module agent framework emphasizing robust agent evaluation and high-quality training data.
* **Key Features**:
    - Modular, extensible agent architecture
    - Test-time reflection and agent voting strategies for improved robustness
    - Demonstrated state-of-the-art results on advanced agentic benchmarks (e.g., GAIA)
    - Available open-source on GitHub, fostering transparency and collaboration
* **Use Cases**: Research, enterprise automation, customizable multi-domain agents

### 4.2 EnvX

* **Description**: EnvX transforms GitHub repositories into autonomous, communicative agents, enabling collaborative operations across multiple projects.
* **Key Features**:
    - Automated understanding and initialization of project repositories
    - Agent-to-Agent (A2A) communication across repositories
    - High performance on tough benchmark tasks (GitTaskBench)
    - Open-source accessibility
* **Use Cases**: DevOps, multi-service orchestration, complex software integrations

### 4.3 LightAgent

* **Description**: A minimalist, open-source agent framework with a focus on core functionality: memory management, tool integration, and structured reasoning (e.g., Tree of Thought).
* **Integration**: Easily integrates with chat platforms for building interactive, self-learning agents. 
* **Best For**: Quick prototyping, chatbots, and lightweight autonomous systems.

## 5. Popular Developer-Centric Frameworks

### 5.1 LangChain

* **Modularity**: Facilitates agentic reasoning, tool usage, memory management, and API integration for multi-step, structured workflows.
* **Strengths**: Best for retrieval-augmented agents, complex logic flows, and rapid customization.

### 5.2 CrewAI

* **Collaboration**: Focuses on role-based agents, enabling teams of diverse agent "personas" to work together on complex tasks.
* **Adoption**: Widely used in customer service, marketing, and collaborative automation scenarios.

### 5.3 OpenAI Agents SDK

* **Lightweight**: Python-based, provider-agnostic SDK supporting over 100 language models.
* **Features**: Tracing, guardrails for reliable workflow automation, and support for multi-agent orchestration.

### 5.4 Google Agent Development Kit (ADK)

* **Ecosystem Integration**: Designed to work seamlessly within Gemini and Vertex AI environments.
* **Hierarchical Agents**: Facilitates building agents that can delegate or collaborate in structured hierarchies.

### 5.5 Microsoft Semantic Kernel

* **Enterprise Integration**: Bridges AI agent functionality with traditional enterprise applications; particularly strong at embedding generative AI into business processes.

## 6. Comparative Analysis

The diversity in AI agent frameworks reflects varying use cases—enterprise workflow automation (Frontier, Agentforce, Security Copilot), developer productivity (Antigravity, ADK), and open research/community needs (Cognitive Kernel-Pro, EnvX, LightAgent). Key trade-offs center on:
* **Usability**: Proprietary platforms (e.g., Frontier, Agentforce) offer seamless integration but may be less flexible for experimentation, while open-source frameworks deliver transparency at the cost of managed infrastructure.
* **Modularity**: Frameworks like LangChain and Cognitive Kernel-Pro support rapid innovation and composition, ideal for experimenting with new kinds of agents.
* **Collaboration**: CrewAI, Agentforce, and EnvX emphasize team-based approaches, critical for complex real-world tasks.

## 7. Current Challenges and Future Trends

### Challenges:
* **Scalability**: Managing large numbers of agents and tasks
* **Interoperability**: Seamless communication between different frameworks and agents
* **Security and Ethics**: Preventing misuse, ensuring user oversight, and building in transparency
* **Usability**: Lowering the barrier for non-experts to create and deploy agents

### Trends:
* Ongoing convergence of agent frameworks with workflow and process automation tools
* Growth of open standards for agent communication, transparency, and evaluation
* Deepening integration with cloud, edge, and on-device compute resources
* Evolution towards more autonomous, self-improving, and cooperative agent populations

## 8. Conclusion

The landscape of AI agent frameworks in 2025 is vibrant, rapidly evolving, and increasingly impactful across industry and society. Enterprises benefit from integrated, secure platforms, while open-source frameworks empower researchers and hobbyists to push the boundaries of agentic intelligence. Future progress will hinge on continued innovation in collaboration, security, evaluation, and ethical deployment.

## 9. References

- [OpenAI Frontier announcement - TechRadar](https://www.techradar.com/pro/openai-introduces-frontier-an-easier-way-to-manage-all-your-ai-agents-in-one-place?utm_source=openai)
- [Project Mariner - Wikipedia](https://en.wikipedia.org/wiki/Project_Mariner?utm_source=openai)
- [Google Antigravity - Wikipedia](https://en.wikipedia.org/wiki/Google_Antigravity?utm_source=openai)
- [Salesforce Agentforce 360 - ITPro](https://www.itpro.com/technology/artificial-intelligence/salesforce-just-launched-a-new-catch-all-platform-to-build-enterprise-ai-agents?utm_source=openai)
- [Microsoft Security Copilot - Axios](https://www.axios.com/2025/03/24/microsoft-ai-agents-cybersecurity?utm_source=openai)
- [Cognitive Kernel-Pro - arXiv](https://arxiv.org/abs/2508.00414?utm_source=openai)
- [EnvX - arXiv](https://arxiv.org/abs/2509.08088?utm_source=openai)
- [LightAgent - arXiv](https://arxiv.org/abs/2509.09292?utm_source=openai)
- [Awesome AI Agents - GitHub](https://github.com/slavakurilyak/awesome-ai-agents)
- [LangChain, CrewAI, OpenAI Agents SDK, Google ADK, Microsoft Semantic Kernel – Various sources]

---

*For further reading, consult the "Awesome AI Agents" repository for an extensive list of frameworks and research on agentic AI.*


### As always, take a look at the trace

https://platform.openai.com/traces

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thanks.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00cc00;">Congratulations on your progress, and a request</h2>
            <span style="color:#00cc00;">You've reached an important moment with the course; you've created a valuable Agent using one of the latest Agent frameworks. You've upskilled, and unlocked new commercial possibilities. Take a moment to celebrate your success!<br/><br/>Something I should ask you -- my editor would smack me if I didn't mention this. If you're able to rate the course on Udemy, I'd be seriously grateful: it's the most important way that Udemy decides whether to show the course to others and it makes a massive difference.<br/><br/>And another reminder to <a href="https://www.linkedin.com/in/eddonner/">connect with me on LinkedIn</a> if you wish! If you wanted to post about your progress on the course, please tag me and I'll weigh in to increase your exposure.
            </span>
        </td>
    </tr>